In [58]:
import pandas as pd

f = open("inputs/footywire player IDs.txt", 'rb')
strings = f.read().decode('UTF-8').split(',')
number = 0
player_id = ''
player_name = ''
player_list = []

for item in strings:
    item = item.replace('"', '')
    if number == 0:
        player_id = item
        number += 1
    elif number == 1:
        player_name = item
        player_list.append({
            'player_id': player_id,
            'player_name': player_name
        })
        number = 0
    
    
pd.DataFrame(player_list).sort_values('player_id')

,player_id,player_name
4603,1,Nathan Bassett
4471,10,Ben Hart
4286,100,Dean Cox
2698,1000,Damian Squire
6158,1001,Paul Sharkey
...,...,...
956,995,Grant Tanner
836,996,Bill Brownless
2994,997,Ewan Thompson
5743,998,Stephen Kernahan


In [59]:
df_footywire_data = pd.read_csv(r'inputs/footywire_player_matches.csv', index_col=0)
df_footywire_data

,Date,Season,Round,Venue,Player,Team,Opposition,Status,Match_id,CP,...,FA,AF,SC,CCL,SCL,SI,MG,TO,ITC,T5
1,2010-03-25,2010,Round 1,MCG,Daniel Connors,Richmond,Carlton,Home,5089,8,...,0,77,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03-25,2010,Round 1,MCG,Daniel Jackson,Richmond,Carlton,Home,5089,11,...,0,85,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-03-25,2010,Round 1,MCG,Brett Deledio,Richmond,Carlton,Home,5089,7,...,2,94,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-03-25,2010,Round 1,MCG,Ben Cousins,Richmond,Carlton,Home,5089,9,...,0,65,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2010-03-25,2010,Round 1,MCG,Trent Cotchin,Richmond,Carlton,Home,5089,8,...,2,65,63,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90416,2019-09-28,2019,Grand Final,MCG,Harrison Himmelberg,GWS,Richmond,Away,9927,3,...,1,65,51,0.0,0.0,3.0,153.0,4.0,2.0,1.0
90417,2019-09-28,2019,Grand Final,MCG,Samuel Reid,GWS,Richmond,Away,9927,5,...,0,47,55,0.0,1.0,1.0,97.0,2.0,5.0,0.0
90418,2019-09-28,2019,Grand Final,MCG,Adam Kennedy,GWS,Richmond,Away,9927,4,...,0,56,55,0.0,1.0,1.0,121.0,0.0,5.0,1.0
90419,2019-09-28,2019,Grand Final,MCG,Shane Mumford,GWS,Richmond,Away,9927,8,...,0,42,62,1.0,1.0,0.0,0.0,1.0,2.0,0.0
